In [1]:
import os

import pandas as pd

from base.learning_strategy import RetrainStrategy
from base.model import Model
from base.training import mse_weighted
from common import Predictor, DataStorage, L2Threshold
from common.model_utils import load_model_from_savemodel
from common.model_utils import load_model_tflite
from common.model_utils import save_model
from simulation.model_simulator import ModelSimulator
from simulation.model_simulator import VIOLATION_COLUMN

if mse_weighted:
    print("mse_weighted loaded")

df_baseline = pd.read_pickle("data/zamg_vienna_hourly.pickle")

THRESHOLD = 1.0
features = ["TL", "P", "RF", "SO"]
df_baseline = df_baseline[features]
df_baseline.dropna(inplace=True)

df_2019_2019 = df_baseline[df_baseline.index.year == 2019]
df_2010_2019 = df_baseline[df_baseline.index.year <= 2019]
initial_df_2010_2019 = df_2010_2019[features].asfreq('H')
initial_df_2019_2019 = df_2019_2019[features].asfreq('H')

sim_data: pd.DataFrame = df_baseline.loc[df_baseline.index.year >= 2020]

threshold = L2Threshold(THRESHOLD, [0], [0])

sim_data

mse_weighted loaded


,TL,P,RF,SO
time,,,,
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0
...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0
2021-12-31 23:20:00,15.2,999.3,59.0,0.0
2021-12-31 23:30:00,15.2,999.3,59.0,0.0


In [2]:
def print_df_stats(name: str, df: pd.DataFrame):
    max_ts = df.index.max()
    min_ts = df.index.min()
    print(f"{name}")
    print(f"---------------------------------------------------")
    print(f"Start date: {min_ts}")
    print(f"End date: {max_ts}")
    print(f"Size of dataframe: {len(df)}")
    for var in features:
        metric: pd.Series = df[var]
        # p75 = metric.quantile(0.75)
        # p95 = metric.quantile(0.95)
        # p99 = metric.quantile(0.99)
        print(f"    {var}: Min={metric.min()}, Mean={metric.mean()}, Median={metric.median()}, Max={metric.max()}")
    print("\n")


print_df_stats("baseline", df_baseline)
print_df_stats("df_2010_2019", df_2010_2019)
print_df_stats("df_2019_2019", df_2019_2019)

baseline
---------------------------------------------------
Start date: 2010-01-01 01:00:00
End date: 2021-12-31 23:50:00
Size of dataframe: 631146
    TL: Min=-14.1, Mean=12.65017117011489, Median=12.5, Max=39.4
    P: Min=955.7, Mean=995.6065873295034, Median=995.6, Max=1025.3
    RF: Min=8.0, Mean=66.88900138267005, Median=68.0, Max=100.0
    SO: Min=0.0, Mean=858.1486882591349, Median=0.0, Max=3600.0


df_2010_2019
---------------------------------------------------
Start date: 2010-01-01 01:00:00
End date: 2019-12-31 23:50:00
Size of dataframe: 525882
    TL: Min=-14.1, Mean=12.66363132160193, Median=12.6, Max=39.4
    P: Min=955.7, Mean=995.5006133822164, Median=995.5, Max=1022.2
    RF: Min=15.0, Mean=67.47230367268703, Median=69.0, Max=100.0
    SO: Min=0.0, Mean=857.6400066935169, Median=0.0, Max=3600.0


df_2019_2019
---------------------------------------------------
Start date: 2019-01-01 00:00:00
End date: 2019-12-31 23:50:00
Size of dataframe: 52560
    TL: Min=-6.0, Mea

In [9]:
# Vienna 2010-2019 simple dense
base_model_2010_2019 = load_model_tflite("models/zamg_vienna_2010_2019_simple_dense")
metadata = base_model_2010_2019.metadata
data_storage = DataStorage(metadata.input_features, metadata.output_features)
data_storage.add_measurement_df(initial_df_2010_2019)
predictor = Predictor(base_model_2010_2019, data_storage)
predictor.update_prediction_horizon(sim_data.index.min())

In [4]:
model_simulator = ModelSimulator(predictor, threshold, sim_data, 600)

results_2010_2019 = model_simulator.run()
results_2010_2019

,TL,P,RF,SO,prediction,error,model,violation
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0,5.1,0.1,zamg_vienna_2010_2019_simple_dense,0
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0,5.032051,0.032051,zamg_vienna_2010_2019_simple_dense,0
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0,4.964102,0.064102,zamg_vienna_2010_2019_simple_dense,0
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0,4.896153,0.003847,zamg_vienna_2010_2019_simple_dense,0
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0,4.828204,0.071796,zamg_vienna_2010_2019_simple_dense,0
...,...,...,...,...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0,15.058929,0.241071,zamg_vienna_2010_2019_simple_dense,0
2021-12-31 23:20:00,15.2,999.3,59.0,0.0,14.990402,0.209598,zamg_vienna_2010_2019_simple_dense,0
2021-12-31 23:30:00,15.2,999.3,59.0,0.0,14.921875,0.278125,zamg_vienna_2010_2019_simple_dense,0
2021-12-31 23:40:00,15.0,999.7,59.0,0.0,14.853349,0.146651,zamg_vienna_2010_2019_simple_dense,0


In [5]:
violations_2010_2019 = results_2010_2019[results_2010_2019[VIOLATION_COLUMN] == True]
print(f"Simulation data: {results_2010_2019.index.min()} - {results_2010_2019.index.max()}")
print(f"Violations: {len(violations_2010_2019)}")
print(f"Measurements: {len(results_2010_2019)}")
print(f"Violations / Measurements = {len(violations_2010_2019) / len(results_2010_2019)}")

Simulation data: 2020-01-01 00:00:00 - 2021-12-31 23:50:00
Violations: 4293
Measurements: 105264
Violations / Measurements = 0.04078317373461012


In [10]:
# Vienna 2019-2019 simple dense
base_model_2019_2019 = load_model_tflite("models/zamg_vienna_2019_2019_simple_dense")
metadata = base_model_2019_2019.metadata
data_storage = DataStorage(metadata.input_features, metadata.output_features)
data_storage.add_measurement_df(initial_df_2010_2019)
predictor = Predictor(base_model_2019_2019, data_storage)
predictor.update_prediction_horizon(sim_data.index.min())

In [2]:
model_simulator = ModelSimulator(predictor, threshold, sim_data, 600)

results_2019_2019 = model_simulator.run()
results_2019_2019

,TL,P,RF,SO,prediction,error,model,violation
2020-01-01 00:00:00,5.0,1012.4,60.0,0.0,5.1,0.1,zamg_vienna_2019_2019_simple_dense,0
2020-01-01 00:10:00,5.0,1012.5,60.0,0.0,4.882985,0.117015,zamg_vienna_2019_2019_simple_dense,0
2020-01-01 00:20:00,4.9,1012.5,61.0,0.0,4.66597,0.23403,zamg_vienna_2019_2019_simple_dense,0
2020-01-01 00:30:00,4.9,1012.5,61.0,0.0,4.448955,0.451045,zamg_vienna_2019_2019_simple_dense,0
2020-01-01 00:40:00,4.9,1012.6,60.0,0.0,4.23194,0.66806,zamg_vienna_2019_2019_simple_dense,0
...,...,...,...,...,...,...,...,...
2021-12-31 23:10:00,15.3,999.2,59.0,0.0,14.849399,0.450601,zamg_vienna_2019_2019_simple_dense,0
2021-12-31 23:20:00,15.2,999.3,59.0,0.0,14.791105,0.408895,zamg_vienna_2019_2019_simple_dense,0
2021-12-31 23:30:00,15.2,999.3,59.0,0.0,14.73281,0.46719,zamg_vienna_2019_2019_simple_dense,0
2021-12-31 23:40:00,15.0,999.7,59.0,0.0,14.674515,0.325485,zamg_vienna_2019_2019_simple_dense,0


In [3]:
violations_2019_2019 = results_2019_2019[results_2019_2019[VIOLATION_COLUMN] == True]
print(f"Simulation data: {results_2019_2019.index.min()} - {results_2019_2019.index.max()}")
print(f"Violations: {len(violations_2019_2019)}")
print(f"Measurements: {len(results_2019_2019)}")
print(f"Violations / Measurements = {len(violations_2019_2019) / len(results_2019_2019)}")

Simulation data: 2020-01-01 00:00:00 - 2021-12-31 23:50:00
Violations: 5861
Measurements: 105264
Violations / Measurements = 0.05567905456756346


In [3]:
models_dir = "models"
learning_strategy = RetrainStrategy()


def train_and_save_model(base_model: Model, model_id: str, train_df: pd.DataFrame) -> Model:
    new_model = learning_strategy.train_model(train_df, base_model.metadata, base_model.model)
    new_model.metadata.uuid = model_id
    save_model(new_model, os.path.join(models_dir, model_id))
    return new_model

In [4]:
# Specialized models by temperature
df_2016_2018 = initial_df_2010_2019[
    (initial_df_2010_2019.index.year <= 2018) & (initial_df_2010_2019.index.year >= 2016)]
length = len(df_2016_2018)
fraction = length // 3
df_sorted = df_2016_2018.sort_values(by=['TL'], ascending=[True])
df_2016_2018_low_temperature: pd.DataFrame = df_sorted.iloc[:fraction]
df_2016_2018_medium_temperature: pd.DataFrame = df_sorted.iloc[fraction:2 * fraction]
df_2016_2018_high_temperature: pd.DataFrame = df_sorted.iloc[2 * fraction:]

print_df_stats("vienna 2016-2018", df_2016_2018)
print_df_stats("vienna 2016-2018 high temperature", df_2016_2018_high_temperature)
print_df_stats("vienna 2016-2018 medium temperature", df_2016_2018_medium_temperature)
print_df_stats("vienna 2016-2018 low temperature", df_2016_2018_low_temperature)

vienna 2016-2018
---------------------------------------------------
Start date: 2016-01-01 00:00:00
End date: 2018-12-31 23:00:00
Size of dataframe: 26304
    TL: Min=-11.0, Mean=12.930060066909975, Median=12.9, Max=38.5
    P: Min=967.1, Mean=995.9371860593546, Median=995.9, Max=1020.4
    RF: Min=17.0, Mean=63.85002281021898, Median=65.0, Max=100.0
    SO: Min=0.0, Mean=872.6569216342253, Median=0.0, Max=3600.0


vienna 2016-2018 high temperature
---------------------------------------------------
Start date: 2016-02-22 13:00:00
End date: 2018-11-01 15:00:00
Size of dataframe: 8768
    TL: Min=17.7, Mean=23.085994525547445, Median=22.4, Max=38.5
    P: Min=973.5, Mean=994.8690921532846, Median=995.1, Max=1011.1
    RF: Min=17.0, Mean=52.33359945255474, Median=52.0, Max=91.0
    SO: Min=0.0, Mean=1654.0987682481752, Median=1357.4999999999998, Max=3600.0


vienna 2016-2018 medium temperature
---------------------------------------------------
Start date: 2016-01-08 11:00:00
End date: 

In [4]:
base_model = load_model_from_savemodel(os.path.join("models", "zamg_vienna_2019_2019_simple_dense"))

In [10]:
train_and_save_model(base_model, "vienna_2016_2018_simple_dense", df_2016_2018)

E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Epoch 1/100
818/818 - 2s - loss: 1.0098 - mean_squared_error: 0.2402 - mean_absolute_error: 0.3388 - root_mean_squared_error: 0.4901 - 2s/epoch - 2ms/step
Epoch 2/100
818/818 - 0s - loss: 0.2885 - mean_squared_error: 0.0769 - mean_absolute_error: 0.2047 - root_mean_squared_error: 0.2773 - 453ms/epoch - 553us/step
Epoch 3/100
818/818 - 0s - loss: 0.2606 - mean_squared_error: 0.0691 - mean_absolute_error: 0.1927 - root_mean_squared_error: 0.2629 - 478ms/epoch - 584us/step
Epoch 4/100
818/818 - 0s - loss: 0.2508 - mean_squared_error: 0.0668 - mean_absolute_error: 0.1887 - root_mean_squared_error: 0.2584 - 461ms/epoch - 564us/step
Epoch 5/100
818/818 - 0s - loss: 0.2449 - mean_squared_error: 0.0653 - mean_absolute_error: 0.1862 - root_mean_squared_error: 0.2556 - 465ms/epoch - 568us/step
Epoch 6/100
818/818 - 0s - loss: 0.2410 - mean_squared_error: 0.0643 - mean_absolute_error: 0.1846 - root_mean_squared_error: 0.2537 - 461ms/epoch - 563us/step
Epoch 7/100
818/818 - 1s - loss: 0.2376 - mea

INFO:tensorflow:Assets written to: models\vienna_2016_2018_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpve1ty89f\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpve1ty89f\assets


In [11]:
train_and_save_model(base_model, "vienna_2016_2018_high_temp_simple_dense", df_2016_2018_high_temperature)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

270/270 - 1s - loss: 2.3728 - mean_squared_error: 0.4650 - mean_absolute_error: 0.5003 - root_mean_squared_error: 0.6819 - 572ms/epoch - 2ms/step
Epoch 2/100
270/270 - 0s - loss: 0.2470 - mean_squared_error: 0.0509 - mean_absolute_error: 0.1731 - root_mean_squared_error: 0.2257 - 156ms/epoch - 579us/step
Epoch 3/100
270/270 - 0s - loss: 0.0793 - mean_squared_error: 0.0164 - mean_absolute_error: 0.0985 - root_mean_squared_error: 0.1282 - 156ms/epoch - 580us/step
Epoch 4/100
270/270 - 0s - loss: 0.0349 - mean_squared_error: 0.0073 - mean_absolute_error: 0.0650 - root_mean_squared_error: 0.0854 - 156ms/epoch - 579us/step
Epoch 5/100
270/270 - 0s - loss: 0.0187 - mean_squared_error: 0.0039 - mean_absolute_error: 0.0471 - root_mean_squared_error: 0.0626 - 173ms/epoch - 641us/step
Epoch 6/100
270/270 - 0s - loss: 0.0112 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0363 - root_mean_squared_error: 0.0489 - 150ms/epoch - 555us/step
Epoch 7/100
270/270 - 0s - loss: 0.0073 - mean_squared

INFO:tensorflow:Assets written to: models\vienna_2016_2018_high_temp_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmp12lmfh2q\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmp12lmfh2q\assets


In [12]:
train_and_save_model(base_model, "vienna_2016_2018_low_temp_simple_dense", df_2016_2018_low_temperature)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

270/270 - 0s - loss: 2.2430 - mean_squared_error: 0.4899 - mean_absolute_error: 0.5053 - root_mean_squared_error: 0.7000 - 461ms/epoch - 2ms/step
Epoch 2/100
270/270 - 0s - loss: 0.2065 - mean_squared_error: 0.0454 - mean_absolute_error: 0.1619 - root_mean_squared_error: 0.2132 - 156ms/epoch - 579us/step
Epoch 3/100
270/270 - 0s - loss: 0.0756 - mean_squared_error: 0.0164 - mean_absolute_error: 0.0979 - root_mean_squared_error: 0.1279 - 172ms/epoch - 637us/step
Epoch 4/100
270/270 - 0s - loss: 0.0347 - mean_squared_error: 0.0075 - mean_absolute_error: 0.0659 - root_mean_squared_error: 0.0867 - 163ms/epoch - 605us/step
Epoch 5/100
270/270 - 0s - loss: 0.0186 - mean_squared_error: 0.0040 - mean_absolute_error: 0.0481 - root_mean_squared_error: 0.0634 - 158ms/epoch - 583us/step
Epoch 6/100
270/270 - 0s - loss: 0.0110 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0369 - root_mean_squared_error: 0.0487 - 174ms/epoch - 645us/step
Epoch 7/100
270/270 - 0s - loss: 0.0070 - mean_squared

INFO:tensorflow:Assets written to: models\vienna_2016_2018_low_temp_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpaprfxnzs\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpaprfxnzs\assets


In [6]:
train_and_save_model(base_model, "vienna_2016_2018_med_temp_simple_dense", df_2016_2018_medium_temperature)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

270/270 - 1s - loss: 2.8732 - mean_squared_error: 0.6047 - mean_absolute_error: 0.5799 - root_mean_squared_error: 0.7776 - 1s/epoch - 4ms/step
Epoch 2/100
270/270 - 0s - loss: 0.2489 - mean_squared_error: 0.0554 - mean_absolute_error: 0.1807 - root_mean_squared_error: 0.2353 - 172ms/epoch - 638us/step
Epoch 3/100
270/270 - 0s - loss: 0.0731 - mean_squared_error: 0.0169 - mean_absolute_error: 0.0992 - root_mean_squared_error: 0.1300 - 172ms/epoch - 637us/step
Epoch 4/100
270/270 - 0s - loss: 0.0302 - mean_squared_error: 0.0071 - mean_absolute_error: 0.0630 - root_mean_squared_error: 0.0842 - 173ms/epoch - 641us/step
Epoch 5/100
270/270 - 0s - loss: 0.0158 - mean_squared_error: 0.0037 - mean_absolute_error: 0.0446 - root_mean_squared_error: 0.0607 - 172ms/epoch - 637us/step
Epoch 6/100
270/270 - 0s - loss: 0.0093 - mean_squared_error: 0.0022 - mean_absolute_error: 0.0337 - root_mean_squared_error: 0.0466 - 177ms/epoch - 655us/step
Epoch 7/100
270/270 - 0s - loss: 0.0060 - mean_squared_er

INFO:tensorflow:Assets written to: models\vienna_2016_2018_med_temp_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpab1grpfp\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpab1grpfp\assets


In [5]:
# Specialized models by season
df_seasonal = initial_df_2010_2019[
    (initial_df_2010_2019.index.year <= 2018) & (initial_df_2010_2019.index.year >= 2016)
    ]
df_spring: pd.DataFrame = df_seasonal[
    (df_seasonal.index.month >= 3) & (df_seasonal.index.month <= 6)
    ]
df_summer: pd.DataFrame = df_seasonal[
    (df_seasonal.index.month >= 6) & (df_seasonal.index.month <= 9)
    ]
df_autumn: pd.DataFrame = df_seasonal[
    (df_seasonal.index.month >= 9) & (df_seasonal.index.month <= 12)
    ]
df_winter: pd.DataFrame = df_seasonal[
    (df_seasonal.index.month == 12) | (df_seasonal.index.month <= 3)
    ]

print_df_stats("vienna 2019-2019", initial_df_2019_2019)
print_df_stats("vienna spring", df_spring)
print_df_stats("vienna summer", df_summer)
print_df_stats("vienna autumn", df_autumn)
print_df_stats("vienna winter", df_winter)

vienna 2019-2019
---------------------------------------------------
Start date: 2019-01-01 00:00:00
End date: 2019-12-31 23:00:00
Size of dataframe: 8760
    TL: Min=-6.0, Mean=13.5362100456621, Median=12.6, Max=37.6
    P: Min=964.9, Mean=994.8188812785388, Median=994.5, Max=1022.2
    RF: Min=16.0, Mean=62.60011415525114, Median=64.0, Max=96.0
    SO: Min=0.0, Mean=916.0503424657534, Median=0.0, Max=3600.0


vienna spring
---------------------------------------------------
Start date: 2016-03-01 00:00:00
End date: 2018-06-30 23:00:00
Size of dataframe: 8784
    TL: Min=-10.8, Mean=15.133709016393443, Median=15.6, Max=34.9
    P: Min=971.8, Mean=993.2802188189053, Median=993.5, Max=1011.2
    RF: Min=18.0, Mean=58.42714025500911, Median=59.0, Max=96.0
    SO: Min=0.0, Mean=1085.5383272616878, Median=0.0, Max=3600.0


vienna summer
---------------------------------------------------
Start date: 2016-06-01 00:00:00
End date: 2018-09-30 23:00:00
Size of dataframe: 8784
    TL: Min=5.2, 

In [15]:
train_and_save_model(base_model, "vienna_2016_2018_spring_simple_dense", df_spring)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

271/271 - 1s - loss: 3.1280 - mean_squared_error: 0.6379 - mean_absolute_error: 0.5910 - root_mean_squared_error: 0.7987 - 969ms/epoch - 4ms/step
Epoch 2/100
271/271 - 0s - loss: 0.7642 - mean_squared_error: 0.1945 - mean_absolute_error: 0.3211 - root_mean_squared_error: 0.4410 - 151ms/epoch - 556us/step
Epoch 3/100
271/271 - 0s - loss: 0.6172 - mean_squared_error: 0.1612 - mean_absolute_error: 0.2839 - root_mean_squared_error: 0.4015 - 157ms/epoch - 579us/step
Epoch 4/100
271/271 - 0s - loss: 0.5723 - mean_squared_error: 0.1511 - mean_absolute_error: 0.2712 - root_mean_squared_error: 0.3887 - 162ms/epoch - 597us/step
Epoch 5/100
271/271 - 0s - loss: 0.5497 - mean_squared_error: 0.1461 - mean_absolute_error: 0.2647 - root_mean_squared_error: 0.3823 - 154ms/epoch - 567us/step
Epoch 6/100
271/271 - 0s - loss: 0.5353 - mean_squared_error: 0.1433 - mean_absolute_error: 0.2606 - root_mean_squared_error: 0.3785 - 159ms/epoch - 587us/step
Epoch 7/100
271/271 - 0s - loss: 0.5237 - mean_squared

INFO:tensorflow:Assets written to: models\vienna_2016_2018_spring_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmptnajh7w6\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmptnajh7w6\assets


In [16]:
train_and_save_model(base_model, "vienna_2016_2018_summer_simple_dense", df_summer)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

271/271 - 0s - loss: 3.3204 - mean_squared_error: 0.7021 - mean_absolute_error: 0.6539 - root_mean_squared_error: 0.8379 - 444ms/epoch - 2ms/step
Epoch 2/100
271/271 - 0s - loss: 1.1355 - mean_squared_error: 0.2791 - mean_absolute_error: 0.4019 - root_mean_squared_error: 0.5283 - 162ms/epoch - 597us/step
Epoch 3/100
271/271 - 0s - loss: 1.0056 - mean_squared_error: 0.2500 - mean_absolute_error: 0.3747 - root_mean_squared_error: 0.5000 - 157ms/epoch - 578us/step
Epoch 4/100
271/271 - 0s - loss: 0.9625 - mean_squared_error: 0.2414 - mean_absolute_error: 0.3662 - root_mean_squared_error: 0.4913 - 157ms/epoch - 579us/step
Epoch 5/100
271/271 - 0s - loss: 0.9350 - mean_squared_error: 0.2356 - mean_absolute_error: 0.3607 - root_mean_squared_error: 0.4854 - 162ms/epoch - 598us/step
Epoch 6/100
271/271 - 0s - loss: 0.9168 - mean_squared_error: 0.2318 - mean_absolute_error: 0.3568 - root_mean_squared_error: 0.4814 - 152ms/epoch - 559us/step
Epoch 7/100
271/271 - 0s - loss: 0.9024 - mean_squared

INFO:tensorflow:Assets written to: models\vienna_2016_2018_summer_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpixklag47\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpixklag47\assets


In [17]:
train_and_save_model(base_model, "vienna_2016_2018_autumn_simple_dense", df_autumn)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

271/271 - 0s - loss: 2.3321 - mean_squared_error: 0.5215 - mean_absolute_error: 0.5247 - root_mean_squared_error: 0.7221 - 460ms/epoch - 2ms/step
Epoch 2/100
271/271 - 0s - loss: 0.5511 - mean_squared_error: 0.1408 - mean_absolute_error: 0.2759 - root_mean_squared_error: 0.3752 - 155ms/epoch - 572us/step
Epoch 3/100
271/271 - 0s - loss: 0.4619 - mean_squared_error: 0.1192 - mean_absolute_error: 0.2496 - root_mean_squared_error: 0.3453 - 157ms/epoch - 579us/step
Epoch 4/100
271/271 - 0s - loss: 0.4359 - mean_squared_error: 0.1133 - mean_absolute_error: 0.2410 - root_mean_squared_error: 0.3365 - 157ms/epoch - 578us/step
Epoch 5/100
271/271 - 0s - loss: 0.4171 - mean_squared_error: 0.1091 - mean_absolute_error: 0.2352 - root_mean_squared_error: 0.3304 - 163ms/epoch - 600us/step
Epoch 6/100
271/271 - 0s - loss: 0.4075 - mean_squared_error: 0.1069 - mean_absolute_error: 0.2317 - root_mean_squared_error: 0.3269 - 157ms/epoch - 579us/step
Epoch 7/100
271/271 - 0s - loss: 0.3945 - mean_squared

INFO:tensorflow:Assets written to: models\vienna_2016_2018_autumn_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpk3y6bfyx\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpk3y6bfyx\assets


In [18]:
train_and_save_model(base_model, "vienna_2016_2018_winter_simple_dense", df_winter)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

269/269 - 0s - loss: 2.8167 - mean_squared_error: 0.6063 - mean_absolute_error: 0.5746 - root_mean_squared_error: 0.7786 - 461ms/epoch - 2ms/step
Epoch 2/100
269/269 - 0s - loss: 0.9833 - mean_squared_error: 0.2559 - mean_absolute_error: 0.3705 - root_mean_squared_error: 0.5059 - 157ms/epoch - 584us/step
Epoch 3/100
269/269 - 0s - loss: 0.8747 - mean_squared_error: 0.2318 - mean_absolute_error: 0.3501 - root_mean_squared_error: 0.4814 - 162ms/epoch - 603us/step
Epoch 4/100
269/269 - 0s - loss: 0.8140 - mean_squared_error: 0.2188 - mean_absolute_error: 0.3379 - root_mean_squared_error: 0.4678 - 148ms/epoch - 550us/step
Epoch 5/100
269/269 - 0s - loss: 0.7779 - mean_squared_error: 0.2118 - mean_absolute_error: 0.3302 - root_mean_squared_error: 0.4603 - 157ms/epoch - 584us/step
Epoch 6/100
269/269 - 0s - loss: 0.7484 - mean_squared_error: 0.2063 - mean_absolute_error: 0.3243 - root_mean_squared_error: 0.4543 - 147ms/epoch - 548us/step
Epoch 7/100
269/269 - 0s - loss: 0.7288 - mean_squared

INFO:tensorflow:Assets written to: models\vienna_2016_2018_winter_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpd3m2vhbl\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpd3m2vhbl\assets


In [6]:
# Specialized models by features
df_features = initial_df_2010_2019[
    (initial_df_2010_2019.index.year <= 2018)
]

quantiles = [0.25, 0.75]
p_quantiles = df_features['P'].quantile(quantiles)
p_quantiles

0.25     991.1
0.75    1000.2
Name: P, dtype: float64

In [7]:
rf_quantiles = df_features['RF'].quantile(quantiles)
rf_quantiles

0.25    56.0
0.75    81.0
Name: RF, dtype: float64

In [8]:
so_quantiles = df_features['SO'].quantile(quantiles)
so_quantiles

0.25       0.0
0.75    1521.0
Name: SO, dtype: float64

In [9]:
min = quantiles[0]
max = quantiles[1]
len(initial_df_2019_2019)

8760

In [10]:
initial_df_2019_2019.corr()["TL"]

TL    1.000000
P    -0.060988
RF   -0.509334
SO    0.412638
Name: TL, dtype: float64

In [11]:
# High humidity, low pressure (storm)
df_high_rf_low_p = df_features[(df_features['P'] <= p_quantiles[min]) & (df_features['RF'] >= rf_quantiles[max])]
print_df_stats("Vienna 2016-2018 High RF Low P", df_high_rf_low_p)

Vienna 2016-2018 High RF Low P
---------------------------------------------------
Start date: 2010-01-01 01:00:00
End date: 2018-12-24 07:00:00
Size of dataframe: 6558
    TL: Min=-10.4, Mean=7.793153400426959, Median=7.9, Max=27.4
    P: Min=955.8, Mean=985.3289265019824, Median=986.7, Max=991.1
    RF: Min=81.0, Mean=88.06602622750839, Median=87.0, Max=100.0
    SO: Min=0.0, Mean=83.74123208295212, Median=0.0, Max=3600.0




In [12]:
# Low humidity, high sun duration (stable)
df_high_so_low_rf = df_features[(df_features['RF'] <= rf_quantiles[min]) & (df_features['SO'] >= so_quantiles[max])]
print_df_stats("Vienna 2016-2018 High SO Low RF", df_high_so_low_rf)

Vienna 2016-2018 High SO Low RF
---------------------------------------------------
Start date: 2010-01-27 08:00:00
End date: 2018-12-31 14:00:00
Size of dataframe: 11836
    TL: Min=-9.5, Mean=21.57225695617889, Median=22.7, Max=39.4
    P: Min=964.8, Mean=995.900278810409, Median=995.9, Max=1018.2
    RF: Min=15.0, Mean=43.69243550749127, Median=45.0, Max=56.0
    SO: Min=1521.0, Mean=3230.823251098344, Median=3599.0, Max=3600.0




In [13]:
# High humidity, low sun duration (dark)
df_high_rf_low_so = df_features[
    (df_features['SO'] <= so_quantiles[min]) & (df_features['RF'] >= rf_quantiles[max])
    ]
print_df_stats("Vienna 2016-2018 Low SO High RF", df_high_rf_low_so)

Vienna 2016-2018 Low SO High RF
---------------------------------------------------
Start date: 2010-01-01 01:00:00
End date: 2018-12-30 14:00:00
Size of dataframe: 19124
    TL: Min=-10.5, Mean=7.588405493969183, Median=7.2, Max=27.4
    P: Min=955.8, Mean=994.6067158195636, Median=994.8, Max=1019.9
    RF: Min=81.0, Mean=87.86404517883288, Median=87.0, Max=100.0
    SO: Min=0.0, Mean=0.0, Median=0.0, Max=0.0




In [14]:
train_and_save_model(base_model, "vienna_2016_2018_storm_simple_dense", df_high_rf_low_p)

E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Epoch 1/100
201/201 - 1s - loss: 3.1496 - mean_squared_error: 0.6717 - mean_absolute_error: 0.6193 - root_mean_squared_error: 0.8196 - 1s/epoch - 7ms/step
Epoch 2/100
201/201 - 0s - loss: 1.0572 - mean_squared_error: 0.2564 - mean_absolute_error: 0.3823 - root_mean_squared_error: 0.5063 - 110ms/epoch - 546us/step
Epoch 3/100
201/201 - 0s - loss: 0.8904 - mean_squared_error: 0.2193 - mean_absolute_error: 0.3471 - root_mean_squared_error: 0.4683 - 112ms/epoch - 555us/step
Epoch 4/100
201/201 - 0s - loss: 0.8321 - mean_squared_error: 0.2062 - mean_absolute_error: 0.3338 - root_mean_squared_error: 0.4541 - 117ms/epoch - 584us/step
Epoch 5/100
201/201 - 0s - loss: 0.8015 - mean_squared_error: 0.1992 - mean_absolute_error: 0.3269 - root_mean_squared_error: 0.4463 - 125ms/epoch - 624us/step
Epoch 6/100
201/201 - 0s - loss: 0.7824 - mean_squared_error: 0.1949 - mean_absolute_error: 0.3227 - root_mean_squared_error: 0.4414 - 141ms/epoch - 702us/step
Epoch 7/100
201/201 - 0s - loss: 0.7685 - mea

INFO:tensorflow:Assets written to: models\vienna_2016_2018_storm_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpoi91s3dk\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpoi91s3dk\assets


In [15]:
train_and_save_model(base_model, "vienna_2016_2018_sunny_simple_dense", df_high_so_low_rf)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

366/366 - 1s - loss: 3.2240 - mean_squared_error: 0.7023 - mean_absolute_error: 0.6227 - root_mean_squared_error: 0.8380 - 893ms/epoch - 2ms/step
Epoch 2/100
366/366 - 0s - loss: 1.2315 - mean_squared_error: 0.2908 - mean_absolute_error: 0.4184 - root_mean_squared_error: 0.5393 - 346ms/epoch - 944us/step
Epoch 3/100
366/366 - 0s - loss: 1.1183 - mean_squared_error: 0.2659 - mean_absolute_error: 0.3990 - root_mean_squared_error: 0.5156 - 341ms/epoch - 932us/step
Epoch 4/100
366/366 - 0s - loss: 1.0639 - mean_squared_error: 0.2538 - mean_absolute_error: 0.3887 - root_mean_squared_error: 0.5038 - 345ms/epoch - 944us/step
Epoch 5/100
366/366 - 0s - loss: 1.0294 - mean_squared_error: 0.2457 - mean_absolute_error: 0.3817 - root_mean_squared_error: 0.4957 - 357ms/epoch - 975us/step
Epoch 6/100
366/366 - 0s - loss: 1.0071 - mean_squared_error: 0.2406 - mean_absolute_error: 0.3771 - root_mean_squared_error: 0.4905 - 340ms/epoch - 928us/step
Epoch 7/100
366/366 - 0s - loss: 0.9906 - mean_squared

INFO:tensorflow:Assets written to: models\vienna_2016_2018_sunny_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpddw6ldh0\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmpddw6ldh0\assets


In [16]:
train_and_save_model(base_model, "vienna_2016_2018_humid_simple_dense", df_high_rf_low_so)

Epoch 1/100


E:\Docs\src\tesi\sense-reduce\common\utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day sin'] = np.sin(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Day cos'] = np.cos(timestamp_s * (2 * np.pi / day)) / normalization_factor
E:\Docs\src\tesi\sense-reduce\common\utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

594/594 - 1s - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - root_mean_squared_error: nan - 1s/epoch - 2ms/step
Epoch 2/100
594/594 - 1s - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - root_mean_squared_error: nan - 625ms/epoch - 1ms/step
Epoch 3/100
594/594 - 1s - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - root_mean_squared_error: nan - 550ms/epoch - 927us/step
Epoch 4/100
594/594 - 1s - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - root_mean_squared_error: nan - 534ms/epoch - 899us/step
Epoch 5/100
594/594 - 1s - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - root_mean_squared_error: nan - 550ms/epoch - 926us/step
Epoch 6/100
594/594 - 1s - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - root_mean_squared_error: nan - 534ms/epoch - 900us/step
Epoch 7/100
594/594 - 1s - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - root_mean_squared_error: nan - 534ms/ep

INFO:tensorflow:Assets written to: models\vienna_2016_2018_humid_simple_dense\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmp48c5hioe\assets


INFO:tensorflow:Assets written to: C:\Users\gabri\AppData\Local\Temp\tmp48c5hioe\assets
